# <center>python小项目使用django的数据库模型</center>

By [青衣极客](https://mp.weixin.qq.com/s/4N7A4UtRl2hvi0KWu1jhvQ)

In 2019-10-06

有时开发一个小项目并不想启动服务，只是在本地运行，但是又有一些数据存储的需求，打算使用数据库。如果直接采用数据库sql语句接口进行操作显得非常麻烦，而且容易出错。现在比较主流的web框架django的数据库模型倒是非常好用，那么有没有办法在自己的小项目中以standalone的形式调用django的数据库模块呢？我在网上查了一些资料，发现中文版本中有一个人提及此事，但是语焉不详，其释放的代码无法正常运行；英文版中有讨论standalone形式的django，但是对于使用数据库模块没有详细表述。在此情况下，本文就讨论一下这个问题。

## 1. 编写数据表字段定义代码

首先我们需要定义数据库中表的字段。在django的web开发中，很自然地在一个app的models.py文件中添加自己的代码。那么在使用standalone模式时最好也保持这个目录结构。有其他网页的帖子上将代码放在一个单独的文件中，我试验过这种方式，在后面执行migrate命令生成数据表时会遇到无法解决的问题。如果不知道我上一句话到底是什么意思，可以忽略，直接按照我下面的方式创建即可。
在搜索目录下创建一个文件夹djangotest, 然后在djangotest目录下创建models.py文件，最后将数据表字段的定义代码写入models.py文件即可。这里演示一个简单的数据表字段定义，该数据表用于存储各个网站的域名、用户名和密码，这个项目的目的是本地记录密码以防止忘记一些重要账户的密码。

In [1]:
!cat ./code/djangotest/models.py


from django.db import models

class Password(models.Model):
    id = models.IntegerField(unique=True, primary_key=True)
    website = models.CharField(max_length=128)
    username = models.CharField(max_length=128)
    pwd = models.CharField(max_length=128)
    time_add = models.DateTimeField(auto_now_add=True, null=True, blank=True)
    time_modify = models.DateTimeField(auto_now=True)

    def __str__(self):
        return 'site={},username={}, pwd={}'.format(
            self.website, self.username, self.pwd) 

    class Meta:
        db_table = 'password_tab'




## 2. 以standalone模式加载django
以standalone模式运行django需要完成两件事：1. 配置环境，2. setup启动。我们想要使用django的数据库模块，这里有两个参数需要配置，INSTALLED_APPS中加入刚才创建的目录djangotest, 即以djangotest作为app的名字；DATABASES中放入数据库的定义，这里直接使用splite3数据库。如果不知道配置的具体含义和形式，可以使用django生成一个web项目，然后查看其settings.py文件中的这两个字段。

In [2]:
import os, sys
sys.path.append('./code/')
import django
from django.db import models
from django.conf import settings
from django.core.management import call_command
# 配置django的环境
INSTALLED_APPS = ['djangotest',]
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': '../../output/db.sqlite3',
    }
}
settings.configure(
    DATABASES=DATABASES, 
    INSTALLED_APPS=INSTALLED_APPS,
    USE_I18N=True)
django.setup()
# 在数据库中创建表
call_command('makemigrations', 'djangotest')
call_command('migrate', 'djangotest')

No changes detected in app 'djangotest'
Operations to perform:
  Apply all migrations: djangotest
Running migrations:
  Applying djangotest.0001_initial... OK


最后运行两个django的命令，makemigrations是根据修改的字段生成的迁移说明py文件，该命令执行完成之后。djangotest目录下会多出一个migrations目录，migrate是根据迁移说明文件创建或修改数据表。如果这两步执行不顺利的话可以检查一下目录结构是否与我上面描述的一致。

## 3. 写入数据

在数据库中创建数据表之后就可以想数据库中写入数据了。这里使用django提供的数据库模型演示一下写入两条简单数据的方法。

In [3]:
from djangotest.models import Password
p = Password(website='example.com', username='hello-01', pwd='12345678')
p.save()
p = Password(website='example.com', username='hello-02', pwd='87654321')
p.save()

## 4. 读取所有数据
写入成功之后，可以将数据读取出来看一下，以检查数据是否跟写入时一致。这里也是调用django的数据库模型进行读取操作。当然django的数据库模型提供了既丰富又方便的数据库读取接口，感兴趣的可以查看django数据库模型的文档。

In [4]:
from djangotest.models import Password
objs = Password.objects.all()
for obj in objs:
    print(obj)
    print(obj.id, obj.website, obj.username, obj.pwd)

site=example.com,username=hello-01, pwd=12345678
1 example.com hello-01 12345678
site=example.com,username=hello-02, pwd=87654321
2 example.com hello-02 87654321


从读取结果来看，数据库操作是成功的。从操作的过程来看也可以发现这是非常简单的，既不用拼接sql语句，也不用检查数据有效性，很多操作都用django自动完成了。

到此在python小项目中使用django的数据库模型就讨论完毕。对此感兴趣并或者还有不明白的朋友可以在评论区留言，希望本文对大家有用。